In [25]:
import pickle
from src.extract_features.deep_learning_embeddings import embedding_vector_with_detect
import cv2
import numpy as np

model_path = "../../data/models/knn_5_cosine_distance_vggface.pkl"

# Load model
with open(model_path, 'rb') as f:
    knn_vgg_face_model = pickle.load(f)


In [40]:
# Load image
image_path = "../data/images_test_predict/022.jpg"
image = cv2.imread(image_path) # Sử dụng cv2 để đọc ảnh


embeddings, facial_area = embedding_vector_with_detect(image_path,
                                          model="VGG-Face",
                                          normalization="base",
                                          detector_backend="mtcnn")

embeddings = np.array(embeddings)


1/1 [==============================] - 0s 29ms/step


In [41]:
distances, indices = knn_vgg_face_model.kneighbors(embeddings)

In [42]:
print(distances)

[[0.10608659 0.11541581 0.12729875 0.12865812 0.13791127]
 [0.19065133 0.19422946 0.19464411 0.2116589  0.21296745]]


In [43]:
print(indices)

[[2048 2025 2035 2028 2027]
 [2004 1992 1962 1974 1967]]


In [46]:
# Tính toán ngưỡng threshold
threshold = np.mean(distances) + np.std(distances)  # có thể tùy chỉnh hằng số 2
print(threshold)

0.20217836199944322


In [47]:
if distances[0][0] <= threshold:
    y_pred = knn_vgg_face_model.predict(embeddings)
    print(y_pred)

['CE049' 'CE048']


In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from PIL import Image

# Load bộ dữ liệu huấn luyện và chuyển đổi sang dạng ma trận
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")

# Tiền xử lý ảnh của người lạ và chuyển đổi sang dạng ma trận
image_path = "unknown_person.jpg"
image = Image.open(image_path)
image = image.resize((64, 64)) # thay đổi kích thước
image = np.array(image) # chuyển đổi sang mảng numpy
X_test = image.reshape(1, -1)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Tính khoảng cách giữa ảnh của người lạ và các điểm dữ liệu trong bộ dữ liệu huấn luyện
knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn.fit(X_train, y_train)
distances, indices = knn.kneighbors(X_test)

# Xác định ngưỡng
threshold = np.mean(distances) + np.std(distances)

# Phân loại người lạ
if distances[0][0] > threshold:
    print("Người này không thuộc nhóm người trong bộ dữ liệu đào tạo.")
else:
    print("Người này thuộc nhóm người trong bộ dữ liệu đào tạo.")
